In [1]:
import numpy as np; np.random.seed(123456)
import pandas as pd
DATA_PATH = "/root/mats/CharacterTraining/data"

In [2]:
large = False

for model in ["gemma", "llama"]:
    path = f"{DATA_PATH}/critiques/outputs/{model}/rephrased"
    path += "_large" if large else ""
    path += ".jsonl"
    outputs = pd.read_json(path, orient="records", lines=True)
    path = f"{DATA_PATH}/critiques/inputs/{model}/rephrased"
    path += "_large" if large else ""
    path += ".jsonl"
    inputs = pd.read_json(path, orient="records", lines=True)
    inputs["response"] = outputs["output"]
    inputs["messages"] = inputs.apply(
        lambda row: [
            {
                "role": "user",
                "content": row["question"][0]["content"]
            },
            {
                "role": "assistant",
                "content": row["response"]
            }
        ], axis=1
    )
    inputs = inputs[["messages"]]
    indices = np.random.permutation(len(inputs))
    inputs = inputs.iloc[indices].iloc[:len(inputs)//2].reset_index(drop=True)
    inputs.to_json(f"{DATA_PATH}/{model}_sft.jsonl", orient="records", lines=True)